## Parse CSPro Dictionary Specification (.DCF) file into dictionary

This notebook contains example code to parse downloaded DCF files (CSPro format dictionary specification files). This is the first step in MAP processing of DHS data obtained in what they call the "hierarchical" format. The DCF files specify a table structure, along with instructions on how to parse the data for these tables out of the data (.DAT) files.

This workbook generates "record spec", "Value spec", and "relationship spec" files from the .DCF file. The most important is the record spec: it specifies the tables and columns that are defined in the file. The file has one row per table column, and contains the necessary information to read the .DAT file and for each row in it, work out what table it belongs to and split the data into the appropriate columns. 

The value spec file specifies for each column of each table, what value or values it may take, and where appropriate, what these values represent if they are somehow coded. 

The relationship spec file records the information from the "relations" section of the DCF. This specifies known or recommended ways of joining tables to produce output (for example) in terms of children, adults, or households. However it doesn't seem to specify all possible links, and so the joiner code can't currently rely entirely on this relationship spec file to construct joins - this part is still experimental and may not be feasible to automate fully.

The parsing is implemented in an external file and this workbook contains a simple loop to call the parser for each DCF file in a directory and write out the results to csv files.

These CSV files are then suitable as input for the code in the DCF_Parser notebook to instruct it how to parse the actual data files (.dat).

In [1]:
from collections import defaultdict
import re
import os
import glob
import csv

In [2]:
# code moved to external file so that FME can also read it
from DCF_Parser_Main import parseDCF

### Parse all DCF files in a directory and write spec to CSV (for db import etc)

In [3]:
# inDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\20160623_Updates\in'
inDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\download'
#outDir =  r'\\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\20160623_Updates\parsed'
outDir = r'\\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\parsed'

In [8]:
# Specify what columns the CSV should have - these must be things that the DCF parser generates
reqfieldnames = ['ItemType', 'FileCode','RecordName','RecordTypeValue','RecordLabel','Name','Label',
                 'Start','Len','Occurrences','ZeroFill', 'DecimalChar', 'Decimal', 'FMETYPE']
valfieldnames = ['FileCode','Name','Value','ValueDesc', 'ValueType']
relfieldnames = ['FileCode', 'RelName', 'PrimaryTable', 'PrimaryLink', 'SecondaryTable', 'SecondaryLink']

inputDCFs = glob.glob(os.path.join(inDir,'*','*.dcf'))

if not os.path.exists(outDir):
    os.makedirs(outDir)
else:
    assert(os.path.isdir(outDir))

for inputDCF in inputDCFs:
    print (inputDCF)
    # parse it!
    parsedDCFItems, parsedDCFRelations = parseDCF(inputDCF, expandRanges="All")
    
    inBase = os.path.extsep.join(os.path.basename(inputDCF).split(os.path.extsep)[:-1])
    outBase = inBase + '.FlatRecordSpec.csv'
    outValsBase = inBase + '.FlatValuesSpec.csv'
    outRelsBase = inBase + '.RelationshipsSpec.csv'
    outFileName = os.path.join(outDir,outBase)
    outValsFileName = os.path.join(outDir, outValsBase)
    outRelsFileName = os.path.join(outDir, outRelsBase)
    
    with open(outFileName, 'w', newline='') as fout, open(outValsFileName, 'w', newline='') as fValsOut, \
            open(outRelsFileName, 'w', newline='') as fRelsOut:
        wri = csv.writer(fout)
        wri.writerow(reqfieldnames)
        wriVals = csv.writer(fValsOut)
        wriVals.writerow(valfieldnames)
        wriRels = csv.writer(fRelsOut)
        wriRels.writerow(relfieldnames)
        for item in parsedDCFItems:
            item['FMETYPE'] = "fme_char({0!s})".format(item['Len'])
            # write the row using the fieldnames given in reqfieldnames
            # not all items have "occurrences", "range_low_value", etc so write blank value if not
            wri.writerow([item[k] if k in item else '' for k in reqfieldnames])
            # write the value sets to a separate file
            if 'Values' in item and len(item['Values'])>0:
                vals = item['Values']
                for val in vals:
                    wriVals.writerow([item['FileCode'],item['Name'], val[0], val[1], val[2]])
        for item in parsedDCFRelations:
            wriRels.writerow([item[k] if k in item else '' for k in relfieldnames])
            

\\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\download\500\500.JOIR71.DCF
Warning, valueset did not seem to match item at line 18829 of file \\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\download\500\500.JOIR71.DCF - please check!
Warning, valueset did not seem to match item at line 30429 of file \\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\download\500\500.JOIR71.DCF - please check!
Parsed 30809 lines into 1909 items
\\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\download\500\500.JOMR71.DCF
Parsed 8302 lines into 508 items
\\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\download\523\523.PKIR71.DCF
Warning, valueset did not seem to match item at line 3557 of file \\map-fs1.ndph.ox.ac.uk\map_data\DHS_Automation\Acquisition\Update_Staging\download\523\523.PKIR71.DCF - please check!
Warning, valueset did not seem to match item at line 33999 of file

In [7]:
item

{'Label': "Reason not using: interferes with body'\x99s processes",
 'Name': 'V3A08T',
 'Start': '138',
 'Len': '1',
 'RecordName': 'REC32',
 'RecordLabel': 'Contraception Knowledge and Use',
 'RecordTypeValue': 'W32',
 'LevelName': 'WOMAN',
 'LevelLabel': 'WOMAN',
 'FileCode': '500.JOIR71',
 'ZeroFill': 'No',
 'DecimalChar': 'No',
 'ItemType': 'Item',
 'Values': [('0', 'No', 'ExplicitValue'),
  ('1', 'Yes', 'ExplicitValue'),
  ('9', 'Missing', 'ExplicitValue'),
  ("' '", 'Not applicable', 'ExplicitValue')],
 'FMETYPE': 'fme_char(1)'}

In [ ]:
# Not part of the parsing - a bit of code to apply before we do anything else to 
# rename downloaded files to include survey id number. Assumes they have been downloaded into 
# subdirectories named with the id number.
allFiles = glob.glob(os.path.join(inDir,'*','*'))
for fn in allFiles:
    if str.lower(fn).find('.zip') != -1:
        continue
    basename = os.path.basename(fn)
    dirname = os.path.dirname(fn)
    idname = os.path.basename(dirname)
    newname = idname+'.'+basename
    newpath = os.path.join(dirname,newname)
    #print fn
    #print newpath
    os.rename(fn,newpath)

### Naive sql generation of output (not used: use FME if loading to DB; this workbook creates CSVs)

None of this is actually used - we primarily work off CSVs and then later loaded those CSVs to PG database en bloc. 

In [ ]:
levelsInserts = []
recordsInserts = []
itemsInserts = []
valuesInserts = []

# i think a straightforward bit of sql formatting will do here, nobody malicious will get chance to run...

# levels is straightforward, just name and label
for name, label in allLevels.iteritems():
    insertLevelsSQL = 'INSERT INTO dhs_levels '\
    '(record_name, record_label)' \
    ' VALUES '\
    '("{0!s}", "{1!s}");'.format(name, label)
    levelsInserts.append(insertLevelsSQL)
    
# records may be country specific, impute this from the presence of a word followed by "specific"
# in the label. E.g. "Country specific", "Survey specific"
for name, label in allRecords.iteritems():
    m = re.match('^\w+ specific', label)
    if m:
        specificText = m.group(0)
    else:
        specificText = "No"
        
    insertRecordsSQL = 'INSERT INTO dhs_records '\
    '(record_name, record_label, c_or_s_specific)' \
    ' VALUES '\
    '("{0!s}", "{1!s}", "{2!s}");'.format(name, label, specificText)
    recordsInserts.append(insertRecordsSQL)
    
# items is the main thingy
for item in allItems:
    insertItemsSQL = 'INSERT INTO dhs_recodes '\
    '(level_id, record_id, recode_id, recode_description, start, len, data_type, item_type, range_low_value, range_high_value)'\
    ' VALUES '\
    '("{0!s}", "{1!s}", "{2!s}", "{3!s}", {4!s}, {5!s}, {6!s}, {7!s}, {8!s}, {9!s});'.format(
        item['LevelName'], item['RecordName'], item['Name'], item['Label'], item['Start'], item['Len'], 
        '"A"', '"B"', # TODO change these
        item['Range_Low_Value'] if item.has_key('Range_Low_Value') else '',
        item['Range_High_Value'] if item.has_key('Range_High_Value') else '',
    )
    itemsInserts.append(insertItemsSQL)
    if item.has_key('Values') and len(item['Values'])>0:
        for valtuple in item['Values']:
            insertValueSQL = 'INSERT INTO dhs_recode_values '\
            '(recode_id, value_code, value_description)' \
            ' VALUES '\
            '("{0!s}", "{1!s}", "{2!s}");'.format(
                item['Name'], valtuple[0], valtuple[1])
            valuesInserts.append(insertValueSQL)
#print insertSQL


